Importing libraries and models

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import sys
import json
from tqdm.notebook import tqdm
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Adding src folder
sys.path.append('../') 

from src.feature_builder_Model2 import(
    extract_features_v8,
    extract_features_v20,
    build_feature_dataframe
)


warnings.filterwarnings('ignore')
SEED = 123 

In [ ]:
# ================================================
# PARTE 1: TRAINING META-MODEL (LEVEL 1)
# ================================================

# Loading training data for the meta-model

print("Caricamento meta-features e target per il training del meta-modello...")

try:
    X_meta_train = pd.read_csv('../data/processed/meta_features_optimal_train.csv')
    y_meta_train = pd.read_csv('../data/processed/train_target.csv').squeeze()

    print(f"Meta-features (X_train L1) caricate. Shape: {X_meta_train.shape}")
    print(f"Target (y_train L1) caricati. Shape: {y_meta_train.shape}")

except FileNotFoundError as e:
    print(f"ERRORE: File non trovato. {e}")
    print("!!! Assicurati di aver prima ESEGUITO il notebook 05_All_Base_Models_Training.ipynb !!!")

Training the Meta-Model

In [ ]:
# Training and saving meta-model
print("--- Training Meta-Modello (Livello 1)... ---")

final_ensemble_model = LogisticRegression(random_state=SEED, max_iter=1000)

final_ensemble_model.fit(X_meta_train, y_meta_train)

print("Meta-Modello allenato.")

META_MODEL_PATH = '../models/final_ensemble_model.joblib'
joblib.dump(final_ensemble_model, META_MODEL_PATH)
print(f"Meta-Modello Finale salvato in: {META_MODEL_PATH}\n")

Generating the Submission

In [ ]:
# Defining the loading data function

def carica_dati_jsonl(file_path):
    """Carica un file .jsonl e lo restituisce como DataFrame."""
    print(f"Caricamento dati da: {file_path}...")
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    df_raw = pd.DataFrame(data)
    return df_raw

print("Funzione 'carica_dati_jsonl' definita localmente.")

In [ ]:
# ================================================
# GENERATE SUBMISSION
# ================================================

print("--- Generazione Submission ---")
print("Caricamento dati di TEST grezzi...")
TEST_DATA_PATH = '../data/raw/test.jsonl'

df_raw_test = carica_dati_jsonl(TEST_DATA_PATH)

print(f"Dati di test caricati. Shape: {df_raw_test.shape}")

In [ ]:
# Generate feature test

print("Generazione feature v8 per il TEST SET...")
X_test_v8 = build_feature_dataframe(df_raw_test, extract_features_v8, is_test_set=True)

print("\nGenerazione feature v20 per il TEST SET...")
X_test_v20 = build_feature_dataframe(df_raw_test, extract_features_v20, is_test_set=True)

print("\nFeature di test generate.")

In [ ]:
print("Caricamento modelli base ottimali (L0) da /models/...")

try:
    
    model_lr_v8 = joblib.load('../models/lr_v8.joblib')
    model_xgb_v20 = joblib.load('../models/xgb_v20.joblib')

    print("Modelli base caricati.")

    X_meta_test = pd.DataFrame()

    print("Generazione predizioni L0 sul test set...")
    X_meta_test['lr_v8'] = model_lr_v8.predict_proba(X_test_v8)[:, 1]
    X_meta_test['xgb_v20'] = model_xgb_v20.predict_proba(X_test_v20)[:, 1]

    print(f"Meta-features di test (X_test L1) create. Shape: {X_meta_test.shape}")

except FileNotFoundError as e:
    print(f"ERRORE: Modello non trovato. {e}")
    print("!!! Assicurati di aver ESEGUITO il notebook 05_All_Base_Models_Training.ipynb !!!")

In [ ]:
print("\n--- Generazione Predizioni Finali (L1) ---")

final_ensemble_model = joblib.load('../models/final_ensemble_model.joblib')

# Making predictions
final_predictions = final_ensemble_model.predict(X_meta_test)

print("Predizioni finali generate.")

# --- Generating submission file ---
submission_df = pd.DataFrame({
    'battle_id': df_raw_test['battle_id'],
    'player_won': final_predictions.astype(int)
})

submission_df.to_csv('../submissions/submission.csv', index=False)

print("\n=================================================")
print("🎉 SUBMISSION.CSV CREATO CON SUCCESSO! 🎉")
print("=================================================")
display(submission_df.head())